In [6]:
import json
from openai import AzureOpenAI
import configparser
import numpy as np

In [7]:
config = configparser.ConfigParser()
config.read('config.ini')
api_key = config.get('default', 'AZURE_OPENAI_KEY')
api_endpoint = config.get('default','AZURE_OPENAI_ENDPOINT')

In [8]:
client = AzureOpenAI(
  azure_endpoint = api_endpoint, 
  api_key=api_key,  
  api_version="2024-02-15-preview"
)

In [9]:
response = client.embeddings.create(
    input = "Esto es un ejemplo para crear embeddings",
    model= "text-embedding-ada-002"
)

print(response.model_dump_json(indent=2))

{
  "data": [
    {
      "embedding": [
        -0.02803691104054451,
        0.0034946671221405268,
        0.01295082550495863,
        -0.013302281498908997,
        0.0008127421606332064,
        0.011346063576638699,
        0.003047057893127203,
        0.00290780165232718,
        -0.005348100792616606,
        -0.029999760910868645,
        -0.005845444276928902,
        0.025013063102960587,
        -0.003929013852030039,
        0.009290376678109169,
        0.0044993008486926556,
        0.013461431488394737,
        0.012665681540966034,
        0.003637238871306181,
        0.031034234911203384,
        -0.01513250544667244,
        -0.020981265231966972,
        0.0007903617224656045,
        0.007334159221500158,
        -0.010152438655495644,
        -0.01158478856086731,
        -0.003207865636795759,
        0.015092717483639717,
        -0.019111253321170807,
        -0.007513202726840973,
        -0.024482563138008118,
        0.03278488293290138,
        -0.009449

In [10]:
embedding = json.loads(response.model_dump_json(indent=2))['data'][0]['embedding']
len(embedding)

1536

In [11]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_embedding(text, model="text-embedding-ada-002"): 
    return client.embeddings.create(input = [text], model=model).data[0].embedding


In [59]:
response_1 = cosine_similarity (get_embedding("puppy"), get_embedding("dog"))
response_1

0.8741718953321239

In [55]:
response_2 = cosine_similarity (get_embedding("I packed my lunch this morning"), get_embedding("Who ate my lunch?"))
response_2

0.8684753226891632

In [56]:
response_3 = cosine_similarity (get_embedding("The pigeon ate my lunch"), get_embedding("Who ate my lunch?"))
response_3

0.894778860779996

In [11]:
text = input('\nEnter a question:\n')


In [17]:
# Create extension config for own data
extension_config = dict(dataSources = [  
        { 
            "type": "AzureCognitiveSearch", 
            "parameters": { 
                "endpoint":'https://aprende-aise-lil.search.windows.net', 
                "key": "ai6CQGkhigkjCEfIbwe9B1RMFCrHzZ8y7s8erCbj2BAzSeDERkO8", 
                "indexName": "lil-experiment-index",
            }
        }]
        )

In [19]:
# Send request to Azure OpenAI model
print("...Sending the following request to Azure OpenAI endpoint...")
print("Request: " + text + "\n")

response = client.chat.completions.create(
    model = 'gpt-4',
    temperature = 0.5,
    max_tokens = 1000,
    messages = [
        {"role": "system", "content": "You are a helpful travel agent"},
        {"role": "user", "content": text}
    ],
    extra_body = extension_config
)


...Sending the following request to Azure OpenAI endpoint...
Request: What's NYC?



BadRequestError: Error code: 400 - {'error': {'message': 'Unrecognized request argument supplied: dataSources. Please contact us through an Azure support request at: https://go.microsoft.com/fwlink/?linkid=2213926 for further questions.', 'type': 'invalid_request_error', 'param': None, 'code': None}}

In [ ]:
# Print response
print("Response: " + response.choices[0].message.content + "\n")

# print data context
print("\nContext information:\n")
context = response.choices[0].message.context
for context_message in context["messages"]:
    context_json = json.loads(context_message["content"])
    print(json.dumps(context_json, indent=2))
